## Generate MSTICpy Config

In [1]:
# Only generate config if Teams
# %%writefile msticpyconfig.yaml
# AzureSentinel:
#   Workspaces:
#     ASIHuntOMSWorkspaceV4:
#       TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
#       WorkspaceId: 52b1ab41-869e-4138-9e40-2a4457f09bf0
#     CyberSecuritySoc:
#       TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
#       WorkspaceId: 8ecf8077-cf51-4820-aadd-14040956f35d
#     Default:
#       TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
#       WorkspaceId: 8ecf8077-cf51-4820-aadd-14040956f35d

## Core MSTICPy initialization for Notebooks

In [2]:
from msticpy.nbtools import nbinit
nbinit.init_notebook(namespace=globals())
qry_prov = QueryProvider("AzureSentinel")

Please wait. Loading Kqlmagic extension...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
qry_prov.connect(WorkspaceConfig())
tables = qry_prov.schema_tables

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Choose A Table To Analzye

In [4]:
import ipywidgets as widgets
from IPython.display import display

print('\nPlease select a table to analyze.\n')
tableDropdown = widgets.Dropdown(
    options=sorted(tables),
    value='OfficeActivity',
    description='Table:',
)
display(tableDropdown)


Please select a table to analyze.



Dropdown(description='Table:', index=231, options=('AACAudit', 'AACHttpRequest', 'AADDomainServicesAccountLogo…

## Choose a Feature(s) and Timeframe to Analyze

In [5]:
df = qry_prov.exec_query(f'{tableDropdown.value} | take 1')
timestamp_col = 'TimeGenerated'

if df.empty:
    print("Table is empty, please select another table.")
else:
    print ("\nWhat kind of features/columns would you like to see be analyzed?\n")
    options = sorted(list(df))
    options.remove(timestamp_col)
    try:
        options.remove('Type')
    except TypeError:
        print('Type does not exist')
    selected_features = nbwidgets.SelectSubset(source_items=options)
    print('\n\nWhat time frame do you want to analyze?\n')
    timeFrame = nbwidgets.QueryTime(units='day', max_before=20, before=5, max_after=1)
    timeFrame.display()

<IPython.core.display.Javascript object>


What kind of features/columns would you like to see be analyzed?




What time frame do you want to analyze?



## Generate Timeseries Anomalies

In [9]:
from time_series_utils import check_kwargs, ts_anomalies_stl

start = (timeFrame.start)
end = (timeFrame.end)
features = selected_features.selected_values

featureDict = {}
anomalyDict = {}

if start == end:
    print("\nPlease make sure the start and end date are distinct\n")
elif len(features) == 0:
    print("\nPlease choose at least one feature\n")
else:
    for feature in features:
        raw_times_series_data = qry_prov.MultiDataSource.get_timeseries_data(
            start=start,
            end=end,
            table=f"{tableDropdown.value}",
            timestampcolumn=timestamp_col,
            aggregatecolumn=f"{feature}",
            aggregatefunction=f"dcount(tostring({feature}))",
            add_query_items=f'|mv-expand {timestamp_col} to typeof(datetime), {feature} to typeof(long)',
        )
        df_time_series = raw_times_series_data[[timestamp_col, f'{feature}']]
        df_time_series = df_time_series.set_index(timestamp_col)
        anomalies = ts_anomalies_stl(df_time_series)
        featureDict[f'{feature}'] = anomalies
        anomalous_timestamps = list(anomalies[anomalies['anomalies']==1][timestamp_col])
        for timestamp in anomalous_timestamps:
            if timestamp not in anomalyDict:
                anomalyDict[timestamp] = [f'{feature}']
            else:
                anomalyDict[timestamp].append(f'{feature}')
#         print(f'Timeseries for {feature} generated')
    print('Timestamp for Anomalous Features\n')
    anomalyDf = pd.DataFrame(anomalyDict.items(), columns=[timestamp_col, 'Anomalous Features'])
    anomalyDf.set_index(timestamp_col)
    display(anomalyDf)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Timestamp for Anomalous Features



,TimeGenerated,Anomalous Features
0,2021-08-10 06:03:03.171821+00:00,"[AffectedItems, ClientIP, ClientIP_, Client_IPAddress, ElevationTime, ItemType, Operation, Recor..."
1,2021-08-11 23:03:03.171821+00:00,"[AppId, ClientIP, ClientIP_, Event_Data, ModifiedProperties, Operation, RecordType]"
2,2021-08-12 00:03:03.171821+00:00,"[ClientIP, ClientIP_, Client_IPAddress, ElevationTime, Event_Data, Folders, ItemType, ModifiedPr..."
3,2021-08-12 01:03:03.171821+00:00,[Folders]
4,2021-08-10 08:03:03.171821+00:00,[Item]
5,2021-08-10 16:03:03.171821+00:00,[Item]
6,2021-08-11 07:03:03.171821+00:00,"[Item, UserType]"
7,2021-08-09 14:03:03.171821+00:00,[Members]
8,2021-08-09 18:03:03.171821+00:00,"[OfficeId, OfficeObjectId, Parameters, SourceRecordId]"
9,2021-08-13 03:03:03.171821+00:00,"[OfficeId, OfficeObjectId, Operation, Parameters, SourceRecordId]"


## Select A Timestamp To Visualize

In [10]:
try:
    timestamp_dropdown = widgets.Dropdown(
        options=sorted(list(anomalyDf[timestamp_col])),
        description='TimeStamp:',
        disabled=False,
    )
    print('\nSelect a timestamp to visualize anomalous features ')
    display(timestamp_dropdown)
except NameError:
    print("Anomaly Dataframe not instantiated yet.")


Select a timestamp to visualize anomalous features 


Dropdown(description='TimeStamp:', options=(Timestamp('2021-08-09 00:03:03.171821+0000', tz='UTC'), Timestamp(…

## Visualize Anomalies

In [13]:
from msticpy.nbtools.timeseries import display_timeseries_anomolies

try:
    anomalous_features = tuple(anomalyDf.loc[anomalyDf[timestamp_col] == timestamp_dropdown.value]['Anomalous Features'])[0]
    for feature in anomalous_features:
        print(f"\nTime Series for {feature}\n")
        display_timeseries_anomolies(data=featureDict[feature], y=feature)
except NameError:
    print("Anomaly Dataframe not instantiated yet.")


Time Series for OfficeId



Loading BokehJS ...


Time Series for OfficeObjectId



Loading BokehJS ...


Time Series for Parameters



Loading BokehJS ...


Time Series for SourceRecordId



Loading BokehJS ...